In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"], axis = 1)
application_df.head()
print(application_df.shape)

(34299, 10)


In [3]:
# Determine the number of unique values in each column.
application_df.nunique(axis=0)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_count = application_df.APPLICATION_TYPE.value_counts()
app_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(app_count[app_count < 725].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_count = application_df.CLASSIFICATION.value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2170        1
C1245        1
C2150        1
C4120        1
C8210        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(class_count[class_count < 1883].index)

# Replace in dataframe
for classification in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(classification,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df. dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True).drop(application_cat, 1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(['IS_SUCCESSFUL'], axis= 1).values
y = application_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 120
hidden_nodes_layer2 = 80

nn = tf.keras.models.Sequential()


# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation = "tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = "tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 120)               5160      
_________________________________________________________________
dense_1 (Dense)              (None, 80)                9680      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 81        
Total params: 14,921
Trainable params: 14,921
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer = "adam", metrics= ["accuracy"])

In [15]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/", exist_ok = True)
checkpoint_path = "checkpoints/AlphabetSoupCharity_Optimization_1.{epoch:02d}.h5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath = checkpoint_path,
    verbose = 1,
    save_weights_only = True,
    save_freq = 5
)

In [16]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 150, callbacks=[cp_callback])

Epoch 1/150
  1/804 [..............................] - ETA: 4:48 - loss: 0.7832 - accuracy: 0.4688
Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5

Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5

Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5

Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5

Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5
 26/804 [..............................] - ETA: 1s - loss: 0.6694 - accuracy: 0.6273  
Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5

Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5

Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5

Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5

Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimiz


Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5
426/804 [==============>...............] - ETA: 0s - loss: 0.5941 - accuracy: 0.7068
Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5

Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5

Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5

Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5

Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5
454/804 [===============>..............] - ETA: 0s - loss: 0.5935 - accuracy: 0.7073
Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5

Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5

Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5

Epoch 00001: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.01.h5

E

 17/804 [..............................] - ETA: 2s - loss: 0.6492 - accuracy: 0.6591
Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5

Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5

Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5

Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5
 41/804 [>.............................] - ETA: 1s - loss: 0.6135 - accuracy: 0.6910
Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5

Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5

Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5

Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5

Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5
 62/804 [=>............................] - ETA: 1s - loss: 0.5979 - accuracy: 0.7036


427/804 [==============>...............] - ETA: 0s - loss: 0.5657 - accuracy: 0.7261
Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5

Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5

Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5

Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5

Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5
452/804 [===============>..............] - ETA: 0s - loss: 0.5652 - accuracy: 0.7263
Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5

Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5

Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5

Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5

Epoch 00002: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.02.h5
47


Epoch 00003: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.03.h5

Epoch 00003: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.03.h5

Epoch 00003: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.03.h5

Epoch 00003: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.03.h5
 48/804 [>.............................] - ETA: 1s - loss: 0.5963 - accuracy: 0.6998
Epoch 00003: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.03.h5

Epoch 00003: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.03.h5

Epoch 00003: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.03.h5

Epoch 00003: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.03.h5

Epoch 00003: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.03.h5
 73/804 [=>............................] - ETA: 1s - loss: 0.5822 - accuracy: 0.7125
Epoch 00003: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.03.h5

E

...


Epoch 00148: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.148.h5

Epoch 00148: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.148.h5

Epoch 00148: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.148.h5

Epoch 00148: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.148.h5
753/804 [===========================>..] - ETA: 0s - loss: 0.5368 - accuracy: 0.7344
Epoch 00148: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.148.h5

Epoch 00148: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.148.h5

Epoch 00148: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.148.h5

Epoch 00148: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.148.h5
777/804 [===========================>..] - ETA: 0s - loss: 0.5367 - accuracy: 0.7346
Epoch 00148: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.148.h5

Epoch 00148: saving model to checkpoints/AlphabetSoupCharity_Optimization_1


Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.149.h5

Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.149.h5
334/804 [===========>..................] - ETA: 1s - loss: 0.5384 - accuracy: 0.7342
Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.149.h5

Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.149.h5

Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.149.h5

Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.149.h5
354/804 [============>.................] - ETA: 1s - loss: 0.5380 - accuracy: 0.7346
Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.149.h5

Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.149.h5

Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.149.h5

Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1


Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.149.h5
729/804 [==========================>...] - ETA: 0s - loss: 0.5354 - accuracy: 0.7373
Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.149.h5

Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.149.h5

Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.149.h5

Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.149.h5
753/804 [===========================>..] - ETA: 0s - loss: 0.5352 - accuracy: 0.7374
Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.149.h5

Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.149.h5

Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.149.h5

Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.149.h5

Epoch 00149: saving model to checkpoints/AlphabetSoupCharity_Optimization_1


Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.150.h5
320/804 [==========>...................] - ETA: 1s - loss: 0.5394 - accuracy: 0.7321
Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.150.h5

Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.150.h5

Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.150.h5

Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.150.h5
342/804 [===========>..................] - ETA: 1s - loss: 0.5394 - accuracy: 0.7321
Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.150.h5

Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.150.h5

Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.150.h5

Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.150.h5

Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1


Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.150.h5
721/804 [=========================>....] - ETA: 0s - loss: 0.5364 - accuracy: 0.7350
Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.150.h5

Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.150.h5

Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.150.h5

Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.150.h5
743/804 [==========================>...] - ETA: 0s - loss: 0.5362 - accuracy: 0.7352
Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.150.h5

Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.150.h5

Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.150.h5

Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1.150.h5

Epoch 00150: saving model to checkpoints/AlphabetSoupCharity_Optimization_1

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5618 - accuracy: 0.7284
Loss: 0.5617958903312683, Accuracy: 0.728396475315094
